In [1]:
from config import *
import datasets
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
import ray
from ray.train.huggingface.transformers import (
    RayTrainReportCallback,
    prepare_trainer,
)
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer
from ray import tune
from ray.tune.search.optuna import OptunaSearch

In [2]:
dataset = datasets.load_from_disk("/data/lab/assignments/proj2/project2/Task1/squad_v2_tokenized_datasets")
dataset = dataset.train_test_split(test_size=test_size)

ray_train_ds = ray.data.from_huggingface(dataset["train"])
ray_eval_ds = ray.data.from_huggingface(dataset["test"])

max_steps_per_epoch = ray_train_ds.count() // (batch_size * num_workers)

In [ ]:
import os
# 指定checkpoint目录
checkpoint_dir = "/root/ray_results/TorchTrainer_2024-05-29_07-12-59/TorchTrainer_e1617_00000_0_2024-05-29_07-12-59/checkpoint_000000/checkpoint"

# 确认 checkpoint 目录存在
if os.path.exists(checkpoint_dir):
    # 加载模型和分词器
    model = T5ForConditionalGeneration.from_pretrained(checkpoint_dir)
    tokenizer = T5Tokenizer.from_pretrained(checkpoint_dir)
    print("Model and tokenizer loaded from checkpoint:", checkpoint_dir)
else:
    raise FileNotFoundError("Checkpoint directory not found: {}".format(checkpoint_dir))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model and tokenizer loaded from checkpoint: /root/ray_results/TorchTrainer_2024-05-29_07-12-59/TorchTrainer_e1617_00000_0_2024-05-29_07-12-59/checkpoint_000000/checkpoint


In [6]:
def train_func(config):
    model = T5ForConditionalGeneration.from_pretrained(model_dir)
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    
    dataset = datasets.load_from_disk("/data/lab/assignments/proj2/project2/Task1/squad_v2_tokenized_datasets")
    dataset = dataset.train_test_split(test_size=test_size)
    
    training_args = Seq2SeqTrainingArguments(
        per_device_train_batch_size=batch_size,
        overwrite_output_dir=True,
        save_total_limit=save_total_limit,
        logging_dir="./logs",
        num_train_epochs=num_train_epochs,
        report_to="none",
        per_device_eval_batch_size=batch_size,
        save_strategy="steps",
        max_steps=20,
        evaluation_strategy="steps",
        output_dir="./checkpoints",
        warmup_steps=2*eval_steps,
        learning_rate=learning_rate,
        logging_steps=eval_steps,
        predict_with_generate=True,
        gradient_accumulation_steps=gradient_accumulation_steps,
        save_steps=eval_steps,
        eval_steps=eval_steps,
    )
   
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
    )
    
    trainer.add_callback(RayTrainReportCallback())
    trainer = prepare_trainer(trainer)
    trainer.train()
    
    return {"eval_loss": trainer.state.log_history[1]['eval_loss']}

In [7]:
tuner = tune.Tuner(
    train_func,
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
    ),
    param_space=search_space,
)
results = tuner.fit()

(train_func pid=487477) You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
(train_func pid=487477) Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
(train_func pid=487477) /root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
(train_func pid=487477)   warnings.warn(
(train_func pid=487477) max_steps is given, it will override any value given in num_

(train_func pid=487477) {'loss': 1.1286, 'grad_norm': 3.6692216396331787, 'learning_rate': 5e-05, 'epoch': 0.0}


 10%|█         | 2/20 [05:16<47:12, 157.36s/it]
(train_func pid=487477) 
  0%|          | 0/163 [00:00<?, ?it/s]
(train_func pid=487477) 
  1%|          | 2/163 [00:05<07:41,  2.87s/it]
(train_func pid=487477) 
  2%|▏         | 3/163 [00:11<10:46,  4.04s/it]
(train_func pid=487477) 
  2%|▏         | 4/163 [00:17<12:20,  4.66s/it]
(train_func pid=487477) 
  3%|▎         | 5/163 [00:22<13:13,  5.02s/it]
(train_func pid=487477) 
  4%|▎         | 6/163 [00:28<13:43,  5.24s/it]
(train_func pid=487477) 
  4%|▍         | 7/163 [00:34<14:02,  5.40s/it]
(train_func pid=487477) 
  5%|▍         | 8/163 [00:39<14:11,  5.49s/it]
(train_func pid=487477) 
  6%|▌         | 9/163 [00:45<14:16,  5.56s/it]
(train_func pid=487477) 
  6%|▌         | 10/163 [00:51<14:30,  5.69s/it]
(train_func pid=487477) 
  7%|▋         | 11/163 [00:57<14:33,  5.75s/it]
(train_func pid=487477) 
  7%|▋         | 12/163 [01:03<14:28,  5.75s/it]
(train_func pid=487477) 
  8%|▊         | 13/163 [01:09<14:24,  5.76s/it]
(train_

(train_func pid=487477) {'eval_loss': 1.095703363418579, 'eval_runtime': 942.821, 'eval_samples_per_second': 2.765, 'eval_steps_per_second': 0.173, 'epoch': 0.0}


(train_func pid=487477) 
                                               t]
100%|██████████| 163/163 [15:37<00:00,  5.69s/it]
                                                 
(train_func pid=487477) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_func_2024-05-29_16-03-19/train_func_f74df_00000_0_lr=0.0001_2024-05-29_16-03-19/checkpoint_000000)
 15%|█▌        | 3/20 [23:34<2:46:20, 587.09s/it]


...

(train_func pid=487477) 
                                                  [A
100%|██████████| 163/163 [15:41<00:00,  5.59s/it]
                                                 
(train_func pid=487477) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_func_2024-05-29_16-03-19/train_func_f74df_00000_0_lr=0.0001_2024-05-29_16-03-19/checkpoint_000008)
 95%|█████████▌| 19/20 [3:13:19<11:57, 717.40s/it]


(train_func pid=487477) {'loss': 0.828, 'grad_norm': 2.116673231124878, 'learning_rate': 0.0, 'epoch': 0.02}


100%|██████████| 20/20 [3:15:52<00:00, 548.07s/it]
(train_func pid=487477) 
  0%|          | 0/163 [00:00<?, ?it/s]
(train_func pid=487477) 
  1%|          | 2/163 [00:05<07:48,  2.91s/it]
(train_func pid=487477) 
  2%|▏         | 3/163 [00:11<11:08,  4.18s/it]
(train_func pid=487477) 
  2%|▏         | 4/163 [00:17<12:43,  4.80s/it]
(train_func pid=487477) 
  3%|▎         | 5/163 [00:23<13:32,  5.14s/it]
(train_func pid=487477) 
  4%|▎         | 6/163 [00:29<14:03,  5.37s/it]
(train_func pid=487477) 
  4%|▍         | 7/163 [00:35<14:17,  5.50s/it]
(train_func pid=487477) 
  5%|▍         | 8/163 [00:41<14:36,  5.66s/it]
(train_func pid=487477) 
  6%|▌         | 9/163 [00:47<14:48,  5.77s/it]
(train_func pid=487477) 
  6%|▌         | 10/163 [00:52<14:48,  5.81s/it]
(train_func pid=487477) 
  7%|▋         | 11/163 [00:58<14:43,  5.81s/it]
(train_func pid=487477) 
  7%|▋         | 12/163 [01:04<14:34,  5.79s/it]
(train_func pid=487477) 
  8%|▊         | 13/163 [01:10<14:26,  5.78s/it]
(tra

(train_func pid=487477) {'eval_loss': 0.6069474220275879, 'eval_runtime': 949.8699, 'eval_samples_per_second': 2.745, 'eval_steps_per_second': 0.172, 'epoch': 0.02}


(train_func pid=487477) 
                                                  [A
100%|██████████| 163/163 [15:44<00:00,  5.72s/it]
                                                 
(train_func pid=487477) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_func_2024-05-29_16-03-19/train_func_f74df_00000_0_lr=0.0001_2024-05-29_16-03-19/checkpoint_000009)
2024-05-29 19:35:09,807	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_func_2024-05-29_16-03-19' in 0.0032s.
2024-05-29 19:35:09,811	INFO tune.py:1039 -- Total run time: 12710.70 seconds (12710.69 seconds for the tuning loop).


In [8]:
print("Best config is:", results.get_best_result().config)

(train_func pid=487477) {'train_runtime': 12705.7604, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.002, 'train_loss': 0.9348451197147369, 'epoch': 0.02}Best config is: {'lr': 0.0001}



100%|██████████| 20/20 [3:31:45<00:00, 635.29s/it]
